In [3]:
import pandas as pd 
import sqlite3
conn = sqlite3.connect("factbook.db")

cursor = conn.cursor()
q = "SELECT* FROM sqlite_master WHERE type ='table';"
cursor.execute(q).fetchall()


[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'facts',
  'facts',
  47,
  'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "area_land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

In [4]:
q = "SELECT* FROM sqlite_master WHERE type ='table';"
pd.read_sql_query(q,conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


# Summary

In [6]:
q2 = "SELECT * FROM facts limit 5"
pd.read_sql_query(q2,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [7]:
q3 = "SELECT min(population) min_pop, max(population) max_pop,min(population_growth) min_growth,max(population_growth) max_growth FROM facts ;"

In [8]:
pd.read_sql_query(q3,conn)

,min_pop,max_pop,min_growth,max_growth
0,0,7256490011,0.0,4.02


In [9]:
q4 = "Select * from facts where population = (select min(population) from facts) "

In [10]:
pd.read_sql_query(q4,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None


# Two outliers found

"World" with max population: 7.2billion
"Antartica" with min population: 0


# Histogram

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(111)
q5 = "Select population, population_growth,birth_rate,death_rate from facts where population != (select max(population) from facts) and population !=(select min(population) from facts)" 
pd.read_sql_query(q5,conn).hist(ax=ax)


/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2885: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  exec(code_obj, self.user_global_ns, self.user_ns)


array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fd228acaf98>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7fd228d44a58>]], dtype=object)

# Which countries have the highest population density?

In [25]:
q6 = "select name, cast(population as float)/cast(area as float) density from facts order by density desc limit 20"

In [26]:
pd.read_sql_query(q6,conn)

,name,density
0,Macau,21168.964286
1,Monaco,15267.500000
2,Singapore,8141.279770
3,Hong Kong,6445.041516
4,Gaza Strip,5191.819444
5,Gibraltar,4876.333333
6,Bahrain,1771.859211
7,Maldives,1319.640940
8,Malta,1310.015823
9,Bermuda,1299.925926


In [29]:
fig2 = plt.figure(figsize = (10,10))
pd.read_sql_query(q6,conn).hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fd228b4bac8>]], dtype=object)

# Which countries have the highest ratios of water to land? 

In [33]:
q7 = "select name, cast(area_water as float)/cast(area_land as float) wlr from facts order by wlr desc limit 20"

In [34]:
pd.read_sql_query(q7,conn)

,name,wlr
0,British Indian Ocean Territory,905.666667
1,Virgin Islands,4.520231
2,Puerto Rico,0.554791
3,"Bahamas, The",0.386613
4,Guinea-Bissau,0.284673
5,Malawi,0.259396
6,Netherlands,0.225710
7,Uganda,0.222922
8,Eritrea,0.164356
9,Liberia,0.156240


# Which countries have more water than land?

In [38]:
q8 = "select * from facts where area_water>area_land"

In [39]:
pd.read_sql_query(q8,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,228,io,British Indian Ocean Territory,54400,60,54340,NaN,NaN,NaN,NaN,NaN
1,247,vq,Virgin Islands,1910,346,1564,103574.0,0.59,10.31,8.54,7.67
